# Python Tutorial

In [ ]:
# Loop

a = 0

if a == 0:
    b = 1
else:
    b = 2
    
# Dictionary 
    
dict = {'Name': 'Zara', 'Age': 7, 'Class': 'First'}

# How to write in python ?

# Constant -> NAME_SURNAME
# Type -> NameSurname
# Global variable, local variable, fonction -> name_surname

# IMPORT

In [1]:
# Path Desktop

%matplotlib inline

from __future__ import print_function
from sklearn.externals import joblib
from pylab import *
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import mixture

import matplotlib.style as ms
ms.use('seaborn-muted')

import math
import time
import glob,os
import pickle
import random
import seaborn
import librosa.display
import numpy as np, scipy, matplotlib.pyplot as plt, sklearn, pandas as pd, librosa, urllib, IPython.display, os.path

plt.rcParams['figure.figsize'] = (14, 5)

# ANNOTATIONS EXCEL

You can directly load the labels file here, if it's already saved

In [2]:
labels = pd.DataFrame.from_csv('labels.csv')

# labels

In [ ]:
label_files = ['044/044.xls','050/050.xls','051/051.xls']
pages = ['hungry','pee','sleepy']

labels = pd.DataFrame()
for label_file in label_files:
    for page in pages: 
        data = pd.read_excel(label_file,page)
        data = data[['Expiration/Inspiration', 'syllable duration', 'syllable start', "comments"]]
        data.rename(inplace=True, columns={'Expiration/Inspiration':'label', 'syllable duration':'dur', 'syllable start':'start', 'comments':'file'})
        labels = pd.concat([labels,data],ignore_index=True)  
        
labels

In [ ]:
# Localisation Colums 0-3 : test.iloc[0:3]

In [ ]:
# Rename labels : 'SI' -> 0, 'IN' -> 1 et 'EX' -> 2. 

def rename_label(label_str):
    if label_str == 'IN':
        return 1
    if label_str == 'EX':
        return 2
    return label_str
    
labels['label']=labels['label'].apply(rename_label)
labels['end']=labels['start']+labels['dur']

# Switch colums

labels = labels[['label', 'dur','start','end','file']]

labels.to_csv('labels.csv')

labels

We can select the first file if we want.

In [ ]:
test_name1 = '01m00d-1(pee&hungry)_050.wav'
test_name2 = '01m13d-1(hungry)_051.wav'
labels[labels.file == test_name2]

# CREATION DATA BASE 

You can directly load the mfcc file here, if it's already saved

In [3]:
all_file_mfcc_norm = np.load('all_file_mfcc_norm.npy').item()
all_file_mfcc = np.load('all_file_mfcc.npy').item()

# all_file_mfcc

You can directly load the labels per frames file here, if it's already saved

In [14]:
all_file_frame_labels = np.load('all_file_frame_labels.npy').item()

# all_file_frame_labels

Extraction de l'audio

In [ ]:
# Windowing parameters for MFCCs

SAMPLING_RATE = 44100
L_WINDOWS = 0.020
L_HOPS = 0.010
L_WINDOW_N = int(round((L_WINDOWS)/(1./SAMPLING_RATE)))
L_HOP_N = int(round((L_HOPS)/(1./SAMPLING_RATE)))
L_WINDOW_N2 = 2**(L_WINDOW_N-1).bit_length() # POWER OF 2 FOR FFT
L_HOP_N2 = 2**(L_WINDOW_N-1).bit_length()
N_MFCC = 13

In [ ]:
baby_states = ['hungry','pee',"sleepy"]
babies = ['044','050','051']

mfcc_scaler = sklearn.preprocessing.StandardScaler()

# Dictionary of mfcc

all_file_mfcc = {}
all_file_mfcc_norm = {}

for baby in babies:
    for baby_state in baby_states: 
        file_path = baby +'/'+baby_state+'/audio/'  
        for audio_file_path in glob.glob(file_path+'*.wav'):
            audio_data = librosa.load(audio_file_path, sr = SAMPLING_RATE)[0]
            # normalize
            audio_data = (audio_data - np.mean(audio_data))/ max(abs(audio_data-np.mean(audio_data)))
            # mfcc extraction
            file_mfcc = librosa.feature.mfcc(audio_data, sr=SAMPLING_RATE, \
                                            n_mfcc=N_MFCC, n_fft = L_WINDOW_N, \
                                            hop_length = L_HOP_N).T
            all_file_mfcc[os.path.basename(audio_file_path)]= file_mfcc
            all_file_mfcc_norm[os.path.basename(audio_file_path)]= mfcc_scaler.fit_transform(file_mfcc)

# We transpose the result to accommodate scikit-learn which assumes that each row is one observation, and each column is one feature dimension

np.save('all_file_mfcc.npy', all_file_mfcc)
np.save('all_file_mfcc_norm.npy', all_file_mfcc_norm) 

# all_file_mfcc

In [ ]:
# extract label for each frame in each file

baby_states = ['hungry','pee',"sleepy"]
babies = ['044','050','051']

# Dictionary of labels

all_file_frame_labels = {}

for baby in babies:
    for baby_state in baby_states:
        file_path = baby +'/'+baby_state+'/audio/'
        for audio_file_path in glob.glob(file_path+'*.wav'):
            audio_file = os.path.basename(audio_file_path)
            file_labels = labels[labels['file']==audio_file]
            # frame labels are 0 unless they're in a IN or EX segment (=1 or 2)
            file_frame_labels = []
            frame_positions = [10+10*x for x in range(len(all_file_mfcc[os.path.basename(audio_file_path)]))]
            for frame_position in frame_positions:
                segment = file_labels[(file_labels['start']<frame_position) &(file_labels['end']>frame_position)]
                if not segment.empty:
                    file_frame_labels.append(segment.label.values[0])
                else:
                    file_frame_labels.append(0)
            all_file_frame_labels[os.path.basename(audio_file_path)]= file_frame_labels
                    
np.save('all_file_frame_labels.npy', all_file_frame_labels) 

Concatenate MFCC files, concatenate label-frame files.
Creating the data base by taking size_dset random consecutive frames.

In [ ]:
# Creation data-set with all contexts and all baby

baby_states = ['hungry','pee',"sleepy"]
babies = ['044','050','051']

concatenate_all_file_mfcc_norm = np.array([]).reshape(0,13)
concatenate_all_file_mfcc = np.array([]).reshape(0,13)
concatenate_all_file_frame_labels = []
audio_files = []

size_dset = 10000

for baby in babies:
    for baby_state in baby_states:
        file_path = baby +'/'+baby_state+'/audio/'
        for audio_file_path in glob.glob(file_path+'*.wav'):
            audio_file = os.path.basename(audio_file_path)
            audio_files = audio_files + [audio_file]
            
# Shuffle files for keeping time relations for HMM

random.shuffle(audio_files)
            
for audio_file in audio_files:
    concatenate_all_file_frame_labels = concatenate_all_file_frame_labels + all_file_frame_labels[audio_file]
    concatenate_all_file_mfcc = np.append(concatenate_all_file_mfcc,all_file_mfcc[audio_file], axis = 0)
    concatenate_all_file_mfcc_norm = np.append(concatenate_all_file_mfcc_norm,all_file_mfcc_norm[audio_file], axis = 0)
    
startpoint = int(floor((len(concatenate_all_file_frame_labels)-size_dset)*random.random()))

labels_dset = copy(concatenate_all_file_frame_labels[startpoint:startpoint+size_dset])
mfcc_dset = copy(concatenate_all_file_mfcc[startpoint:startpoint+size_dset])
mfcc_dset_norm = copy(concatenate_all_file_mfcc_norm[startpoint:startpoint+size_dset])

print(shape(labels_dset),shape(mfcc_dset))

# TEST ON A FILE 

In [ ]:
SAMPLING_RATE = 44100

audio_file_path = '044/hungry/audio/00m12d-1(hungry)_044.wav'

audio_data = librosa.load(audio_file_path, sr = SAMPLING_RATE)[0]
audio_data = (audio_data - np.mean(audio_data))/ max(abs(audio_data-np.mean(audio_data)))

IPython.display.Audio(audio_data, rate=SAMPLING_RATE)

In [ ]:
# Plot WaveSound

plt.figure()

plt.subplot(2, 1, 1)
librosa.display.waveplot(audio_data, SAMPLING_RATE)
plt.ylabel('Sound Amplitude')

# Plot excel annotations

plt.subplot(2, 1, 2)
plt.plot(all_file_frame_labels[os.path.basename(audio_file_path)])
plt.ylabel('Label')
plt.xlabel('Annotated Frames')
axes = plt.gca()
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])
plt.tight_layout()

# SVM creation and hyperparameters

Choose hyperparameters C and Gamma with tenfold Cross-Validation

In [ ]:
C_range = np.logspace(-3, 3, 7)
gamma_range = np.logspace(-3, 3, 7)
param_grid = {'gamma' : gamma_range, 'C' : C_range}
cv = sklearn.model_selection.KFold(n_splits=10, shuffle=False, random_state=None)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(mfcc_dset_norm, labels_dset)
print("The best parameters are " + str(grid.best_params_) + " with a score of " + str(grid.best_score_))

Finer tuning on basis 2 for Gamma

In [ ]:
C_range = [1.0]
gamma_range = [0.015625,0.03125,0.0625,0.125,0.5]
param_grid = {'gamma' : gamma_range, 'C' : C_range}
cv = sklearn.model_selection.KFold(n_splits=10, shuffle=False, random_state=None)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(mfcc_dset_norm, labels_dset)
print("The best parameters are " + str(grid.best_params_) + " with a score of " + str(grid.best_score_))

C = grid.best_params_['C']
gamma = grid.best_params_['gamma']

Fixing C and gamma. C is always 1 and gamma is often 0.0625 or 0.125. We are taking C = 1 and gamma = 0.1.

In [ ]:
C = 1
gamma = 0.1

Create svm classifer model object 

In [ ]:
model_svm = sklearn.svm.SVC(C = C, gamma = gamma, probability=True)

# GMM creation

In [4]:
def train_gmm3(features_train,labels_train,n_components):
    M0 = mixture.GaussianMixture(n_components = n_components, max_iter = 50, n_init = 20, init_params = 'kmeans')
    M1 = mixture.GaussianMixture(n_components = n_components, max_iter = 50, n_init = 20, init_params = 'kmeans')
    M2 = mixture.GaussianMixture(n_components = n_components, max_iter = 50, n_init = 20, init_params = 'kmeans')
    #
    label_0 = features_train[np.where(labels_train == 0)]
    label_1 = features_train[np.where(labels_train == 1)]
    label_2 = features_train[np.where(labels_train == 2)]
    #
    M0.fit(label_0)
    M1.fit(label_1)
    M2.fit(label_2)
    #
    model = [M0,M1,M2]
    return model

In [5]:
def predict_gmm3(features_test,model):
    M0 = model[0]
    M1 = model[1]
    M2 = model[2]
    # log likelihood per labels
    llhood_0 = M0.score_samples(features_test)
    llhood_1 = M1.score_samples(features_test)
    llhood_2 = M2.score_samples(features_test)
    llhood = [llhood_0,llhood_1,llhood_2]
    lhood = np.exp(llhood)
    # add exits and entries states
    a = np.zeros((1,lhood.shape[1]))
    a[a < 1e-100] = 1e-100
    b = lhood
    b[b < 1e-100] = 1e-100
    llhood = np.log(concatenate((a,b,a)))
    # predict !
    state_seq = np.zeros(len(llhood_0))
    state_seq [np.where((llhood_0 > llhood_1)&(llhood_0 > llhood_2))] = 0
    state_seq [np.where((llhood_1 > llhood_0)&(llhood_1 > llhood_2))] = 1
    state_seq [np.where((llhood_2 > llhood_0)&(llhood_2 > llhood_1))] = 2
    return llhood, state_seq

In [6]:
def gmm3_score(features_test,labels_test,model):
    state_seq = predict_gmm3(features_test,model)[1]
    new_score = float(len(np.squeeze(np.where(state_seq == labels_test))))/float(len(labels_test))
    return new_score

# HMM Creation

Transforming the outputs of SVM into a probability distribution over classes with Platt scaling extended for the multiclasse case by Wu et al. (A,B) are estimated by maximization of maximum likelihood, probabilities given by 5fold cross validation.     

See "Probability Estimates for Multi-class Classification by Pairwise Coupling" (2004)

In [7]:
def log_lhood_svm(features,model):
    predicted_proba_labels = model.predict_proba(features)
    predicted_proba_labels = predicted_proba_labels.transpose()
    # Add entry and exit states for viterbi
    a = np.zeros((1,predicted_proba_labels.shape[1]))
    a[a < 1e-100] = 1e-100
    b = predicted_proba_labels
    b[b < 1e-100] = 1e-100
    # take the log-probability
    log_lhood = np.log(concatenate((a,b,a)))
    return log_lhood

Training the Transition probability Matrix for Viterbi decoding

In [8]:
def log_trans_matrix(states):
    transition_prob_matrix = np.zeros((3,3))
    for test in range(len(states)-1):
        state1 = states[test]
        state2 = states[test+1]
        transition_prob_matrix[state1,state2] = transition_prob_matrix[state1,state2] + 1
    # Normalize
    row_sums = transition_prob_matrix.sum(axis=1)
    row_sums = row_sums[:, np.newaxis]
    transition_prob_matrix = transition_prob_matrix/(row_sums*np.ones((1,3))+1e-100)
    # Add entry and exit states into transition matrix
    transition_prob_matrix2 = np.zeros((5,5))
    transition_prob_matrix2[1:4,1:4] = transition_prob_matrix
    transition_prob_matrix2[0,1:4] = np.ones((1,3))/np.ones((1,3)).sum()
    transition_prob_matrix2[transition_prob_matrix2 < 1e-100] = 1e-100
    # return the log matrix transition
    log_trans = np.log(transition_prob_matrix2)
    return log_trans

Filtering the classifications decisions using the viterbi algorithm 

In [9]:
def log_viterbi(log_lhood, log_trans):
    # Parameters and initialization 
    numStates, numPts = log_lhood.shape
    delta = np.zeros(numStates)
    psi = np.zeros((numPts,numStates))
    # State 0 : 'entry state', state 1 : 'SI', state 2 : 'IN', state 3 : 'EX', state 4 :'exit state'
    for i in range(1, numStates-1):
        delta[i] = log_trans[0,i] + log_lhood[i,0]
        psi[0,i] = 0
    # Recursion
    for t in range(1,numPts):
        deltabefore = delta
        for i in range(1,numStates-1):
            temp = deltabefore[1:numStates-1] + log_trans[1:numStates-1,i]
            maxDelta = np.amax(temp)
            index = np.argmax(temp)
            delta[i] = maxDelta + log_lhood[i,t]
            psi[t,i] = index + 1
    # termination
    state_seq = np.zeros(numPts+2)
    state_seq[numPts+1] = numStates-1
    temp = delta[1:numStates-1] + log_trans[1:numStates-1,numStates-1]
    maxDelta = np.amax(temp)
    index = np.argmax(temp)
    state_seq[numPts] = index + 1
    #Backtracking
    for t in range(numPts-1,0,-1):
        state_seq[t] = psi[t,int(state_seq[t+1])]
    state_seq[0] = 0
    state_seq = state_seq[1:numPts+1]
    state_seq = state_seq -1
    return state_seq

Accuracy score after viterbi filtering : model is SVM or GMM

In [10]:
def svm_viterbi_score(features_test,labels_test,log_trans,model):
    log_lhood = log_lhood_svm(features_test,model)
    state_seq = log_viterbi(log_lhood, log_trans)
    new_score = float(len(np.squeeze(np.where(state_seq == labels_test))))/float(len(labels_test))
    return new_score

In [11]:
def gmm3_viterbi_score(features_test,labels_test,log_trans,model):
    log_lhood = predict_gmm3(features_test,model)[0]
    state_seq = log_viterbi(log_lhood, log_trans)
    new_score = float(len(np.squeeze(np.where(state_seq == labels_test))))/float(len(labels_test))
    return new_score

# Training and testing for the two classifier

You can directly load the two svm model file here, if it's already saved

In [ ]:
# model_svm = joblib.load('model_svm.pkl')
# model_svm

Separate the data base in a training base (90% of the data base) and a test base (10% of the data base).

In [ ]:
# Separate in training and data base

number_train = int(round((0.90*len(labels_dset))))

labels_dset_train = copy(labels_dset[:number_train])
labels_dset_test = copy(labels_dset[number_train:])
mfcc_dset_train = copy(mfcc_dset[:number_train])
mfcc_dset_test = copy(mfcc_dset[number_train:])

mfcc_dset_train_norm = copy(mfcc_dset_norm[:number_train])
mfcc_dset_test_norm = copy(mfcc_dset_norm[number_train:])

Train the svm classifier:

In [ ]:
start = time.time()
model_svm.fit(mfcc_dset_train_norm, labels_dset_train)
end = time.time()
print(end - start)

# Save model
joblib.dump(model_svm, 'model_svm.pkl') 

model_svm

Train the gmm classifier: 

In [ ]:
n_components = 5

start = time.time()
model_gmm3_norm = train_gmm3(mfcc_dset_train_norm,labels_dset_train,n_components)
model_gmm3 = train_gmm3(mfcc_dset_train,labels_dset_train,n_components)
end = time.time()
print(end - start)

model_gmm3

Train the log transition Matrix for Viterbi decoding

In [ ]:
log_trans = log_trans_matrix(labels_dset_train)

Compute the accuracy score of the two classifier on the train data : frame per frame and fpf + viterbi

In [ ]:
score = {}

score_svm = model_svm.score(mfcc_dset_train_norm, labels_dset_train)
score_svm_vtb = svm_viterbi_score(mfcc_dset_train_norm,labels_dset_train,log_trans,model_svm)
score_gmm3 = gmm3_score(mfcc_dset_train,labels_dset_train,model_gmm3)
score_gmm3_vtb = gmm3_viterbi_score(mfcc_dset_train,labels_dset_train,log_trans,model_gmm3)
score_gmm3_norm = gmm3_score(mfcc_dset_train_norm,labels_dset_train,model_gmm3_norm)
score_gmm3_vtb_norm = gmm3_viterbi_score(mfcc_dset_train_norm,labels_dset_train,log_trans,model_gmm3_norm)

score['score_svm']= score_svm
score['score_svm_vtb'] = score_svm_vtb
score['score_gmm3']= score_gmm3
score['score_gmm3_vtb'] = score_gmm3_vtb
score['score_gmm3_norm']= score_gmm3_norm
score['score_gmm3_vtb_norm'] = score_gmm3_vtb_norm
score

Finally, compute the accuracy score of the classifier on the test data: frame per frame and fpf + viterbi

In [ ]:
score = {}

score_svm = model_svm.score(mfcc_dset_test_norm, labels_dset_test)
score_svm_vtb = svm_viterbi_score(mfcc_dset_test_norm,labels_dset_test,log_trans,model_svm)
score_gmm3 = gmm3_score(mfcc_dset_test,labels_dset_test,model_gmm3)
score_gmm3_vtb = gmm3_viterbi_score(mfcc_dset_test,labels_dset_test,log_trans,model_gmm3)
score_gmm3_norm = gmm3_score(mfcc_dset_test_norm,labels_dset_test,model_gmm3_norm)
score_gmm3_vtb_norm = gmm3_viterbi_score(mfcc_dset_test_norm,labels_dset_test,log_trans,model_gmm3_norm)

score['score_svm']= score_svm
score['score_svm_vtb'] = score_svm_vtb
score['score_gmm3']= score_gmm3
score['score_gmm3_vtb'] = score_gmm3_vtb
score['score_gmm3_norm']= score_gmm3_norm
score['score_gmm3_vtb_norm'] = score_gmm3_vtb_norm
score

# TEST ON A FILE FOR SVM

Comparison between audio, excel annotations, classification by frame per frame, segmentation per fpf + viterbi.  

In [ ]:
plt.rcParams['figure.figsize'] = (14, 14)

SAMPLING_RATE = 44100
audio_file_path = '044/hungry/audio/00m12d-1(hungry)_044.wav'
audio_data = librosa.load(audio_file_path, sr = SAMPLING_RATE)[0]
audio_data = (audio_data - np.mean(audio_data))/ max(abs(audio_data-np.mean(audio_data)))

# Plot WaveSound

plt.figure()
plt.subplot(8, 1, 1)
librosa.display.waveplot(audio_data, SAMPLING_RATE)
plt.ylabel('Sound Amplitude')

# Plot excel annotations

plt.subplot(8, 1, 2)
plt.plot(all_file_frame_labels[os.path.basename(audio_file_path)])
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Annotated Frames')
axes = plt.gca()
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])

# Plot classification by frame per frame (SVM)

file_mfcc_norm = all_file_mfcc_norm[os.path.basename(audio_file_path)]
predicted_labels = model_svm.predict(file_mfcc_norm)

plt.subplot(8, 1, 3)
plt.plot(predicted_labels)
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Esteemed Frames by SVM')
axes = plt.gca()
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])

# Plot classification by frame per frame (SVM) + viterbi 

log_lhood = log_lhood_svm(file_mfcc_norm,model_svm)
state_seq = log_viterbi(log_lhood, log_trans)

plt.subplot(8, 1, 4)
plt.plot(state_seq)
axes = plt.gca()
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Esteemed Frames by SVM + vtb')
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])

# Plot classification by frame per frame (GMM)

file_mfcc = all_file_mfcc[os.path.basename(audio_file_path)]
log_lhood, predicted_labels = predict_gmm3(file_mfcc,model_gmm3)

plt.subplot(8, 1, 5)
plt.plot(predicted_labels)
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Esteemed Frames by GMM')
axes = plt.gca()
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])

# Plot classification by frame per frame (GMM) + viterbi 

state_seq = log_viterbi(log_lhood, log_trans)

plt.subplot(8, 1, 6)
plt.plot(state_seq)
axes = plt.gca()
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Esteemed Frames by GMM + vtb')
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])
plt.tight_layout()

# Plot classification by frame per frame (GMM) normalized

log_lhood, predicted_labels = predict_gmm3(file_mfcc_norm,model_gmm3_norm)

plt.subplot(8, 1, 7)
plt.plot(predicted_labels)
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Esteemed Frames by GMM normalized')
axes = plt.gca()
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])

# Plot classification by frame per frame (GMM) + viterbi normalized

state_seq = log_viterbi(log_lhood, log_trans)

plt.subplot(8, 1, 8)
plt.plot(state_seq)
axes = plt.gca()
plt.ylabel('Label')
plt.yticks(arange(3),('SI', 'IN', 'EX'))
plt.xlabel('Esteemed Frames by GMM + vtb normalized')
axes.set_xlim([0,len(all_file_frame_labels[os.path.basename(audio_file_path)])])
plt.tight_layout()

# Extraction of states duration and start with labels 

In [ ]:
state_seq

In [ ]:
def states_description(state_seq):
    # Init
    start_states = []
    duration_states = []
    type_states = []
    #
    duration_state = 0
    start_state = 0
    frame_before = state_seq[0]
    for i in range(len(state_seq)):
        if (frame_before != state_seq[i]):
            start_states = start_states + [start_state]
            duration_states = duration_states + [duration_state]
            type_states = type_states + [frame_before]
            #
            start_state = start_state + duration_state + 10
            duration_state = 0
        duration_state = duration_state + 10
        frame_before = state_seq[i]
    # Add the last one
    start_states = start_states + [start_state]
    duration_states = duration_states + [duration_state]
    type_states = type_states + [frame_before]
    # Creation a DataFrame
    df_states = pd.DataFrame({'Start' : start_states,\
                         'Duration' : duration_states,\
                         'label' : type_states})
    return df_states

In [ ]:
df_states = states_description(state_seq)

def rename_label(label_str):
    if label_str == 1:
        return 'IN'
    if label_str == 2:
        return 'EX'
    if label_str == 0:
        return 'SI'
    return label_str
    
df_states['label']=df_states['label'].apply(rename_label)

df_states

# CROSS VALIDATION AND ACCURACY

ten fold cross validation : model_type = 1 for svm and svm+vtb, model_type = 2 for gmm and gmm+vtb

In [12]:
def accuracy_cv(features,labels,model_type,C,gamma,n_components):
    # Initialization
    kf = KFold(n_splits=10)
    scores_model = np.array([])
    scores_model_vtb = np.array([])
    # Choice of algorithm
    if (model_type == 1):
        algo = 'svm'
    else:
        algo = 'gmm'
    # Cross_validation
    for train_index, test_index in kf.split(features):
        features_train, features_test = features[train_index], features[test_index]
        labels_train, labels_test = labels[train_index], labels[test_index]
        if (model_type == 1):
            model = sklearn.svm.SVC(C = C, gamma = gamma, probability=True)
            model.fit(features_train, labels_train)
            score_model = model.score(features_test, labels_test)
            log_trans = log_trans_matrix(labels_train)
            score_model_vtb = svm_viterbi_score(features_test,labels_test,log_trans,model)
        else:
            model = train_gmm3(features_train,labels_train,n_components)
            score_model = gmm3_score(features_test,labels_test,model)
            log_trans = log_trans_matrix(labels_train)
            score_model_vtb = gmm3_viterbi_score(features_test,labels_test,log_trans,model)
        scores_model = np.append(scores_model,score_model)
        scores_model_vtb = np.append(scores_model_vtb,score_model_vtb)
    # display scores
    print('Accuracy ' + algo +  ' with ten fold cross validation :')
    print(scores_model)
    accuracy_model = scores_model.mean()
    print("Accuracy: %0.2f (+/- %0.2f)" % (accuracy_model, scores_model.std() * 2))
    print('\n')
    print('Accuracy ' + algo + ' + viterbi with ten fold cross validation :')
    print(scores_model_vtb)
    accuracy_model_vtb = scores_model_vtb.mean()
    print("Accuracy: %0.2f (+/- %0.2f)" % (accuracy_model_vtb, scores_model_vtb.std() * 2))
    print('\n')
    return accuracy_model, accuracy_model_vtb

# CROSS VALIDATION FOR DIFFERENTS DATASETS

In [15]:
# Creation 6 data-set with 1 contexts and 1 baby

baby_states = ['hungry','pee',"sleepy"]
babies = ['044','050']

labels_dsets = {}
mfcc_dsets = {}
mfcc_dsets_norm = {}

size_dset = 40000

for baby in babies:
    for baby_state in baby_states:
        concatenate_all_file_mfcc = np.array([]).reshape(0,13)
        concatenate_all_file_mfcc_norm = np.array([]).reshape(0,13)
        concatenate_all_file_frame_labels = []
        audio_files = []
        file_path = baby +'/'+baby_state+'/audio/'
        for audio_file_path in glob.glob(file_path+'*.wav'):
            audio_file = os.path.basename(audio_file_path)
            audio_files = audio_files + [audio_file]
        # Shuffle files for keeping time relations for HMM   
        random.shuffle(audio_files)
        for audio_file in audio_files:
            concatenate_all_file_frame_labels = concatenate_all_file_frame_labels + all_file_frame_labels[audio_file]
            concatenate_all_file_mfcc = np.append(concatenate_all_file_mfcc,all_file_mfcc[audio_file], axis = 0)
            concatenate_all_file_mfcc_norm = np.append(concatenate_all_file_mfcc_norm,all_file_mfcc_norm[audio_file], axis = 0)
        # Taking size_dset frames    
        startpoint = int(floor((len(concatenate_all_file_frame_labels)-size_dset)*random.random()))
        labels_dset = copy(concatenate_all_file_frame_labels[startpoint:startpoint+size_dset])
        mfcc_dset = copy(concatenate_all_file_mfcc[startpoint:startpoint+size_dset])
        mfcc_dset_norm = copy(concatenate_all_file_mfcc_norm[startpoint:startpoint+size_dset])
        labels_dsets[baby + '-' + baby_state] = labels_dset
        mfcc_dsets[baby + '-' + baby_state] = mfcc_dset
        mfcc_dsets_norm[baby + '-' + baby_state] = mfcc_dset_norm


In [ ]:
C = 1
gamma = 0.1

accuracies = {}

for baby in babies:
    for baby_state in baby_states:
        name_dataset = baby + '-' + baby_state
        mfcc_dset = mfcc_dsets[name_dataset]
        mfcc_dset_norm = mfcc_dsets_norm[name_dataset]
        labels_dset = labels_dsets[name_dataset]
        # SVM + SVM/Vtb
        accuracy_model, accuracy_model_vtb = accuracy_cv(mfcc_dset_norm,labels_dset,1,C,gamma,5)
        accuracies[baby + '-' + baby_state + '-' + 'svm'] = accuracy_model        
        accuracies[baby + '-' + baby_state + '-' + 'svm+vtb'] = accuracy_model_vtb
        # GMM5 + GMM5/vtb
        accuracy_model, accuracy_model_vtb = accuracy_cv(mfcc_dset,labels_dset,2,C,gamma,5)
        accuracies[baby + '-' + baby_state + '-' + 'gmm5'] = accuracy_model        
        accuracies[baby + '-' + baby_state + '-' + 'gmm5+vtb'] = accuracy_model_vtb
        # GMM5 + GMM5/vtb normalized
        accuracy_model, accuracy_model_vtb = accuracy_cv(mfcc_dset_norm,labels_dset,2,C,gamma,5)
        accuracies[baby + '-' + baby_state + '-' + 'gmm5 norm'] = accuracy_model        
        accuracies[baby + '-' + baby_state + '-' + 'gmm5+vtb norm'] = accuracy_model_vtb
        # GMM20 + GMM20/vtb
        accuracy_model, accuracy_model_vtb = accuracy_cv(mfcc_dset,labels_dset,2,C,gamma,20)
        accuracies[baby + '-' + baby_state + '-' + 'gmm20'] = accuracy_model        
        accuracies[baby + '-' + baby_state + '-' + 'gmm20+vtb'] = accuracy_model_vtb
        # GMM20 + GMM20/vtb normalized
        accuracy_model, accuracy_model_vtb = accuracy_cv(mfcc_dset_norm,labels_dset,2,C,gamma,20)
        accuracies[baby + '-' + baby_state + '-' + 'gmm20 norm'] = accuracy_model        
        accuracies[baby + '-' + baby_state + '-' + 'gmm20+vtb norm'] = accuracy_model_vtb

In [ ]:
babies = ['044','050']
baby_states = ['hungry','pee',"sleepy"]
types_algo = ['svm','svm+vtb','gmm5','gmm5+vtb','gmm5 norm','gmm5+vtb norm','gmm20','gmm20+vtb','gmm20 norm','gmm20+vtb norm']

index = ['44', '44', '44', '50','50', '50']

df = pd.DataFrame({'Context' : ['Hung', 'Pee', 'Sleepy', 'Hung','Pee', 'Sleepy'],\
                          'svm (%)' : np.random.randn(6),\
                   'svm+vtb (%)' : np.random.randn(6),\
                  'gmm5 (%)' : np.random.randn(6),\
                  'gmm5+vtb (%)' : np.random.randn(6),\
                  'gmm5 norm (%)' : np.random.randn(6),\
                  'gmm5+vtb norm (%)' : np.random.randn(6),\
                  'gmm20 (%)' : np.random.randn(6),\
                  'gmm20+vtb (%)' : np.random.randn(6),\
                  'gmm20 norm (%)' : np.random.randn(6),\
                  'gmm20+vtb norm (%)' : np.random.randn(6)}, index = index)

df = df[['Context','svm (%)','svm+vtb (%)','gmm5 (%)','gmm5+vtb (%)','gmm5 norm (%)','gmm5+vtb norm (%)','gmm20 (%)','gmm20+vtb (%)','gmm20 norm (%)','gmm20+vtb norm (%)']]

i = 0

for baby in babies:
    for baby_state in baby_states:
        for type_algo in types_algo:
            if (type_algo == 'svm'):
                df.iat[i,1] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'svm+vtb'):
                df.iat[i,2] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm5'):
                df.iat[i,3] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm5+vtb'):
                df.iat[i,4] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm5 norm'):
                df.iat[i,5] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm5+vtb norm'):
                df.iat[i,6] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm20'):
                df.iat[i,7] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm20+vtb'):
                df.iat[i,8] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm20 norm'):
                df.iat[i,9] = accuracies[baby + '-' + baby_state + '-' + type_algo]
            elif (type_algo == 'gmm20+vtb norm'):
                df.iat[i,10] = accuracies[baby + '-' + baby_state + '-' + type_algo]
        i = i + 1

df